In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import cv2
import os
import PIL

%matplotlib inline

/home/death-reaper/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ori_img_path = "./Digimon"
dest_img_path = "./resized_digimon_images"
noalpha_img_path = "./resized_digimon_black/"

In [3]:
def preprocess_images():
    
    if not os.path.exists(dest_img_path):
        os.mkdir(dest_img_path)
        
    if not os.path.exists(noalpha_img_path):
        os.mkdir(noalpha_img_path)
        
    for digimon_img in os.listdir(ori_img_path):
        d_image = cv2.imread(os.path.join(ori_img_path, digimon_img))
        if d_image is not None:
            d_image = cv2.resize(d_image, (128, 128))
            cv2.imwrite(os.path.join(dest_img_path, digimon_img), d_image)
            
            resized_image = Image.open(os.path.join(dest_img_path, digimon_img))
            if resized_image.mode == 'RGBA':
                resized_image.load()
                bg = Image.new('RGB', resized_image.size, (0, 0, 0))
                bg.paste(resized_image, mask=resized_image.split()[3])
                bg.save(os.path.join(noalpha_img_path, digimon_img.split('.')[0] + '.jpg'), 'JPEG')
            else:
                resized_image.convert("RGB")
                resized_image.save(os.path.join(noalpha_img_path, digimon_img.split('.')[0] + '.jpg'), 'JPEG')
    
    print("Done preprocessing images")

In [4]:
preprocess_images()

Done preprocessing images


In [5]:
def generate_tensor(digimon_list):
    images_tensor = tf.convert_to_tensor(digimon_list, dtype=tf.string)
    image_queue = tf.train.slice_input_producer([images_tensor])
    return image_queue
    
def process_images_to_batches():
    
    digimon_list = []
    for digimon_image in os.listdir(noalpha_img_path):
        digimon_list.append(os.path.join(noalpha_img_path, digimon_image))
        
    image_queue = generate_tensor(digimon_list)
    content_file = tf.read_file(image_queue[0])
    
    decoded_images = tf.image.decode_jpeg(content_file)
    decoded_images = tf.image.random_brightness(image = decoded_images, max_delta=0.1)
    decoded_images = tf.image.random_flip_left_right(image=decoded_images)
    decoded_images = tf.image.random_contrast(decoded_images, lower = 0.1, upper=0.9)
    decoded_images = tf.image.resize_images(decoded_images, [128, 128])
    decoded_images.set_shape([128, 128, 3])
    
    decoded_images =tf.cast(decoded_images, dtype=tf.float32)
    decoded_images /=255.0
    
    batches = tf.train.shuffle_batch([decoded_images], 
                                     64, 
                                     capacity = 200 + 3*64, 
                                     num_threads=4, min_after_dequeue = 200)
    
    
    return batches, len(digimon_list)

In [6]:
def generator(image_input, noise_dim, is_train, reuse=False):
    
    with tf.variable_scope("gen", reuse = tf.AUTO_REUSE) as scope:
        if reuse:
            scope.reuse_vaiables()
            
        gen_weight0 = tf.get_variable("gen_weight0", 
                                      shape=[noise_dim, 4*4*512],
                                      dtype = tf.float32, 
                                      initializer=tf.truncated_normal_initializer(stddev=0.02))
        gen_bias0 = tf.get_variable("gen_bias0", 
                                    shape = [4*4*512], 
                                    dtype = tf.float32,
                                    initializer = tf.constant_initializer(0.0))
        flat_conv0 = tf.add(tf.matmul(image_input, gen_weight0), gen_bias0, name = "flat_conv0")
        
        ## 4*4*512
        gen_conv1 = tf.reshape(flat_conv0, [-1, 4, 4, 512], name = "gen_conv1")
        gen_bias1 = tf.contrib.layers.batch_norm(gen_conv1, 
                                                 epsilon = 1e-5, 
                                                 updates_collections=None, 
                                                 is_training=is_train, 
                                                 decay = 0.9, 
                                                 scope="gen_bias1", 
                                                 activation_fn = tf.nn.relu)
        
        ## 8*8*256
        gen_conv2 = tf.layers.conv2d_transpose(gen_bias1, 
                                               256, 
                                               kernel_size = [5, 5], 
                                               strides = [2, 2], 
                                               padding = "SAME", 
                                               kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), 
                                               name = "gen_conv2")
        gen_bias2 = tf.contrib.layers.batch_norm(gen_conv2, 
                                                 decay = 0.9, 
                                                 epsilon = 1e-5, 
                                                 updates_collections = None, 
                                                 is_training = is_train, 
                                                 scope = "gen_bias2", 
                                                 activation_fn = tf.nn.relu)
        
        ## 16*16*128
        gen_conv3 = tf.layers.conv2d_transpose(gen_bias2, 
                                               128, 
                                               kernel_size = [5, 5], 
                                               strides = [2, 2], 
                                               padding = "SAME", 
                                               kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), 
                                               name = "gen_conv3")
        gen_bias3 = tf.contrib.layers.batch_norm(gen_conv3, 
                                                 decay = 0.9, 
                                                 epsilon = 1e-5, 
                                                 updates_collections = None, 
                                                 is_training=is_train, 
                                                 scope="gen_bias3", 
                                                 activation_fn = tf.nn.relu)
        
        
        ## 32*32*64
        gen_conv4 = tf.layers.conv2d_transpose(gen_bias3, 
                                               64, 
                                               kernel_size = [5, 5], 
                                               strides = [2, 2], 
                                               padding = "SAME", 
                                               kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), 
                                               name = "gen_conv4")
        gen_bias4 = tf.contrib.layers.batch_norm(gen_conv4, 
                                                 decay = 0.9, 
                                                 epsilon = 1e-5, 
                                                 updates_collections = None, 
                                                 is_training=is_train, 
                                                 scope="gen_bias4", 
                                                 activation_fn = tf.nn.relu)
        
        ## 64*64*32
        gen_conv5 = tf.layers.conv2d_transpose(gen_bias4, 
                                               32, 
                                               kernel_size = [5, 5], 
                                               strides = [2, 2], 
                                               padding = "SAME", 
                                               kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), 
                                               name = "gen_conv5")
        gen_bias5 = tf.contrib.layers.batch_norm(gen_conv5, 
                                                 decay = 0.9, 
                                                 epsilon = 1e-5, 
                                                 updates_collections = None, 
                                                 is_training=is_train, 
                                                 scope="gen_bias5", 
                                                 activation_fn = tf.nn.relu)
        
        ## 128*128*3
        gen_conv6 = tf.layers.conv2d_transpose(gen_bias5, 
                                               3, 
                                               kernel_size = [5, 5], 
                                               strides = [2, 2], 
                                               padding = "SAME", 
                                               kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), 
                                               name = "gen_conv6")
        gen_bias6 = tf.contrib.layers.batch_norm(gen_conv6, 
                                                 decay = 0.9, 
                                                 epsilon = 1e-5, 
                                                 updates_collections = None, 
                                                 is_training=is_train, 
                                                 scope="gen_bias6", 
                                                 activation_fn = tf.nn.tanh)
        
        return gen_bias6

In [7]:
def discriminator(input_image, is_train, reuse = False):
    with tf.variable_scope("disc", reuse=tf.AUTO_REUSE) as scope:
        if reuse:
            scope.reuse_variables()
            
        ## 64*64*64
        disc_conv0 = tf.layers.conv2d(input_image, 
                                      64, 
                                      kernel_size = [5, 5], 
                                      strides = [2, 2], 
                                      padding = "SAME", 
                                      kernel_initializer = tf.truncated_normal_initializer(stddev = 0.02), 
                                      name = "disc_conv0")
        
        disc_bias0 = tf.contrib.layers.batch_norm(disc_conv0, 
                                                  decay = 0.9, 
                                                  epsilon = 1e-5, 
                                                  updates_collections = None, 
                                                  is_training = is_train, 
                                                  scope = "disc_bias0", 
                                                 activation_fn = tf.nn.relu)
        
        ## 32*32*128
        disc_conv1 = tf.layers.conv2d(disc_bias0, 
                                      128, 
                                      kernel_size = [5, 5], 
                                      strides = [2, 2], 
                                      padding = "SAME", 
                                      kernel_initializer = tf.truncated_normal_initializer(stddev = 0.02), 
                                      name = "disc_conv1")
        
        disc_bias1 = tf.contrib.layers.batch_norm(disc_conv1, 
                                                  decay = 0.9, 
                                                  epsilon = 1e-5, 
                                                  updates_collections = None, 
                                                  is_training = is_train, 
                                                  scope = "disc_bias1", 
                                                 activation_fn = tf.nn.relu)
        
        ## 16*16*256
        disc_conv2 = tf.layers.conv2d(disc_bias1, 
                                      256, 
                                      kernel_size = [5, 5], 
                                      strides = [2, 2], 
                                      padding = "SAME", 
                                      kernel_initializer = tf.truncated_normal_initializer(stddev = 0.02), 
                                      name = "disc_conv2")
        
        disc_bias2 = tf.contrib.layers.batch_norm(disc_conv2, 
                                                  decay = 0.9, 
                                                  epsilon = 1e-5, 
                                                  updates_collections = None, 
                                                  is_training = is_train, 
                                                  scope = "disc_bias2", 
                                                 activation_fn = tf.nn.relu)
        
        ## 8*8*512
        disc_conv3 = tf.layers.conv2d(disc_bias2, 
                                      512, 
                                      kernel_size = [5, 5], 
                                      strides = [2, 2], 
                                      padding = "SAME", 
                                      kernel_initializer = tf.truncated_normal_initializer(stddev = 0.02), 
                                      name = "disc_conv3")
        
        disc_bias3 = tf.contrib.layers.batch_norm(disc_conv3, 
                                                  decay = 0.9, 
                                                  epsilon = 1e-5, 
                                                  updates_collections = None, 
                                                  is_training = is_train, 
                                                  scope = "disc_bias3", 
                                                 activation_fn = tf.nn.relu)
        
        ## 4*4*1024
        disc_conv4 = tf.layers.conv2d(disc_bias3, 
                                      1024, 
                                      kernel_size = [5, 5], 
                                      strides = [2, 2], 
                                      padding = "SAME", 
                                      kernel_initializer = tf.truncated_normal_initializer(stddev = 0.02), 
                                      name = "disc_conv4")
        
        disc_bias4 = tf.contrib.layers.batch_norm(disc_conv4, 
                                                  decay = 0.9, 
                                                  epsilon = 1e-5, 
                                                  updates_collections = None, 
                                                  is_training = is_train, 
                                                  scope = "disc_bias4", 
                                                 activation_fn = tf.nn.relu)
        
        
        ## zeroth fully connected layer
        dimen = int(np.prod(disc_bias4.get_shape()[1:]))
        disc_fc0 = tf.reshape(disc_bias4, shape=[-1, dimen], name = "disc_fc0")
        
        disc_fc_weight0 = tf.get_variable("disc_fc_weight0", 
                                       shape=[disc_fc0.shape[-1], 1024], 
                                       dtype=tf.float32, 
                                       initializer = tf.truncated_normal_initializer(stddev=0.02))
        disc_fc_bias0 = tf.get_variable("disc_fc_bias0", 
                                        shape=[1024], 
                                        dtype=tf.float32, 
                                        initializer = tf.constant_initializer(0.0))
        disc_fc_act0 = tf.nn.relu(tf.matmul(disc_fc0, disc_fc_weight0) + disc_fc_bias0)
        
        ## first fully connected layer        
        disc_fc_weight1 = tf.get_variable("disc_fc_weight1", 
                                       shape=[1024, 128], 
                                       dtype=tf.float32, 
                                       initializer = tf.truncated_normal_initializer(stddev=0.02))
        disc_fc_bias1 = tf.get_variable("disc_fc_bias1", 
                                        shape=[128], 
                                        dtype=tf.float32, 
                                        initializer = tf.constant_initializer(0.0))
        disc_fc_act1 = tf.nn.relu(tf.matmul(disc_fc_act0, disc_fc_weight1) + disc_fc_bias1)
        
        ## second fully connected layer        
        disc_fc_weight2 = tf.get_variable("disc_fc_weight2", 
                                       shape=[128, 1], 
                                       dtype=tf.float32, 
                                       initializer = tf.truncated_normal_initializer(stddev=0.02))
        disc_fc_bias2 = tf.get_variable("disc_fc_bias2", 
                                        shape=[1], 
                                        dtype=tf.float32, 
                                        initializer = tf.constant_initializer(0.0))
        disc_fc_act2 = tf.matmul(disc_fc_act1, disc_fc_weight2) + disc_fc_bias2
        
        return disc_fc_act2

In [8]:
def train_digiGan():
    tf_sess = tf.Session()
    batches = 64
    noise_dim = 100
    
    real_image = tf.placeholder(tf.float32, shape = [None, 128, 128, 3], name = "real_image")
    noise_input = tf.placeholder(tf.float32, shape = [None, noise_dim], name = "noise_input")
    is_train = tf.placeholder(tf.bool, name = "is_train")
    
    gen_func = generator(noise_input, noise_dim, is_train)
    disc_func = discriminator(real_image, is_train)
    disc_gen_func = discriminator(gen_func, is_train, reuse = True)
    
    gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen_func, 
                                                                      labels = tf.ones_like(disc_gen_func)))
    
    disc_X_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_func, 
                                                                         labels = tf.fill([batches, 1], 0.9)))
    disc_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen_func, 
                                                                           labels=tf.zeros_like(disc_gen_func)))
    disc_loss = disc_X_loss+disc_fake_loss
    
    trainable_vars = tf.trainable_variables()
    
    g_vars = [var for var in trainable_vars if 'gen' in var.name]
    d_vars = [var for var in trainable_vars if 'disc' in var.name]
    
    with tf.variable_scope(tf.get_variable_scope(), reuse = tf.AUTO_REUSE):
        disc_train_X = tf.train.AdamOptimizer(0.001).minimize(disc_X_loss, var_list = d_vars)
        disc_train_fake = tf.train.AdamOptimizer(0.001).minimize(disc_fake_loss, var_list=d_vars)
        gen_train = tf.train.AdamOptimizer(0.001).minimize(gen_loss, var_list=g_vars)
        
    disc_count_real = tf.placeholder(tf.float32)
    disc_count_fake = tf.placeholder(tf.float32)
    gen_count = tf.placeholder(tf.float32)
    disc_after_train_fake = tf.reduce_mean(discriminator(generator(noise_input, noise_dim, is_train), is_train))
    disc_after_train_real = tf.reduce_mean(discriminator(real_image, is_train))
    images = generator(noise_input, noise_dim, is_train)

    tf.summary.scalar("Generator_loss", gen_loss)
    tf.summary.scalar("Discriminator_fake_loss", disc_fake_loss)
    tf.summary.scalar("Discriminator_real_loss", disc_X_loss)
    tf.summary.scalar("discriminator_count_real", disc_count_real)
    tf.summary.scalar("discriminator_count_fake", disc_count_fake)
    tf.summary.scalar("generator_count", gen_count)
    tf.summary.scalar("discriminator_values_of_generated", disc_after_train_fake)
    tf.summary.scalar("discriminator_values_of_real", disc_after_train_real)
    tf.summary.image("generated_fake_images", images)

    merged = tf.summary.merge_all()
    logdir = "tensorboard/digimonGan/"
    fileWriter = tf.summary.FileWriter(logdir, graph = tf_sess.graph)
    
    saver = tf.train.Saver()
    tf_sess.run(tf.global_variables_initializer())
    tf_sess.run(tf.local_variables_initializer())
    gen_loss = 0.0
    discLossReal, discLossFake = 1.0, 1.0
    discFakeCount, discRealCount, genCount = 0.0, 0.0, 0.0
    
    images_batch, images_num = process_images_to_batches()
    tot_batches = int(images_num/batches)
    
    print(logdir + "\n\n")
    print(merged + "\n\n")
    print(str(disc_X_loss) + "\n\n")
    print(str(disc_fake_loss) + "\n\n")
    print(str(disc_loss) + "\n\n")
    print(str(trainable_vars) + "\n\n")
    print(str(gen_func) + "\n\n")
    print(str(disc_gen_func) + "\n\n")
    print(str(d_vars) + "\n\n")
    print(str(g_vars) + "\n\n")
    print(str(disc_after_train_fake) + "\n\n")
    print(str(disc_after_train_real) + "\n\n")
    print("total no. of batches: " + str(tot_batches) + "\n\n")
    print("total no. of images: " + str(images_num)+ "\n\n")
    
    print("Training started......\n\n")
    for i in range(5000):
        
        for batch in range(tot_batches):
            d_iters = 5
            g_iters = 1
            noiseInp = np.random.uniform(-1.0, 1.0, size=[batches, noise_dim]).astype(np.float32)
            
            for j in range(d_iters):
                ori_batch = tf_sess.run(images_batch)
                
                if discLossReal > 0.45:
                    _, discLossReal, discLossFake, genLoss = tf_sess.run([disc_train_X, 
                                                              disc_X_loss, 
                                                              disc_fake_loss, 
                                                              gen_loss], 
                                                              feed_dict = {real_image: ori_batch, 
                                                                           is_train: True})
                    discRealCount += 1
                
                if discLossFake > 0.6:
                    _, discLossReal, discLossFake, genLoss = tf_sess.run([disc_train_fake, 
                                                                  disc_X_loss, 
                                                                  disc_fake_loss, 
                                                                  gen_loss], 
                                                                  feed_dict = {real_image: ori_batch, 
                                                                               is_train: True, 
                                                                               noise_input: noiseInp})
                    discFakeCount += 1
                    
            for j in range(g_iters):
                if genLoss > 0.5:
                    _, discLossReal, discLossFake, genLoss = tf_sess.run([gen_train, 
                                                                          disc_X_loss, 
                                                                          disc_fake_loss, 
                                                                          gen_loss], 
                                                                          feed_dict={real_image: ori_batch, 
                                                                                     is_train: True, 
                                                                                     noise_input: noiseInp})
                    genCount += 1
                    
        if i%50 == 0:
            if not os.path.exists("./digimonModels"):
                os.mkdir("./digimonModels")
                
            sample_noise = np.random.uniform(-1.0, 1.0, size = [batches, noise_dim]).astype(np.float32)
            test_img = tf_sess.run(gen_func, feed_dict = {noise_input: sample_noise, is_train: False})
            
            saved_at = saver.save(tf_sess, "./digimonModels/pretrained_digi_gan.ckpt", global_step = i)
            print("Image saved at: {0}".format(saved_at))
            print("gen_loss: {0}, \ndiscLossReal: {1}, \ndiscLossFake: {2}".format(genLoss, discLossReal, discLossFake))

In [ ]:
print("Let's generate some digimons")

train_digiGan()

Let's generate some digimons
tensorboard/digimonGan/


Tensor("add_1:0", shape=(), dtype=string)
Tensor("Mean_1:0", shape=(), dtype=float32)


Tensor("Mean_2:0", shape=(), dtype=float32)


Tensor("add:0", shape=(), dtype=float32)


[<tf.Variable 'gen/gen_weight0:0' shape=(100, 8192) dtype=float32_ref>, <tf.Variable 'gen/gen_bias0:0' shape=(8192,) dtype=float32_ref>, <tf.Variable 'gen/gen_bias1/beta:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'gen/gen_conv2/kernel:0' shape=(5, 5, 256, 512) dtype=float32_ref>, <tf.Variable 'gen/gen_conv2/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'gen/gen_bias2/beta:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'gen/gen_conv3/kernel:0' shape=(5, 5, 128, 256) dtype=float32_ref>, <tf.Variable 'gen/gen_conv3/bias:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'gen/gen_bias3/beta:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'gen/gen_conv4/kernel:0' shape=(5, 5, 64, 128) dtype=float32_ref>, <tf.Variable 'gen/gen_conv4/bias:0' shape=(6